## Ortho Prediction Software Developing by krish:

#### main / Global Variable use in this program

In [81]:
#Blastp = /usr/bin/blastp
Blastp = "blastp"
Blastp_data =  './blastp_data/'
cpu_count = 1 # Default value
#genome = None
genome = ""
infinite_loop = 70
inflation_factor = 1.4
#mode = None
mode = None
Cluster_out = "./cluster_out/"
#threshold_score : 0
threshold_score =0
Save_raw_blastp_score = False
Score_file = "./score_file"
Species = "./species/"
verbose = False

## 1. Read_Species_List()

In [5]:
import glob
species = "./species/"
def Read_Species_List(pr=1):
    global species
    """ If pr is 1, it will print "Species_List". """ 
    read_species =glob.glob(species+"/*")   
    selected_species_dic = {}
    backward_selected_species_dic = {}
    number = 0
    for i, species in enumerate(sorted(read_species), start=1):
        selected_species_dic[i] = species.split('/')[-1]
        backward_selected_species_dic[species.split('/')[-1]] = i 
        #if pr == 1 :
            #print (str(i)+".", species.split('/')[-1])
        number = i
    return selected_species_dic, backward_selected_species_dic, number
Read_Species_List()

({1: 'species\\AAE',
  2: 'species\\CAC',
  3: 'species\\ECO',
  4: 'species\\ECU',
  5: 'species\\HIN',
  6: 'species\\LLA',
  7: 'species\\SCE',
  8: 'species\\SPO',
  9: 'species\\SPY',
  10: 'species\\SYN',
  11: 'species\\TMA',
  12: 'species\\YPE'},
 {'species\\AAE': 1,
  'species\\CAC': 2,
  'species\\ECO': 3,
  'species\\ECU': 4,
  'species\\HIN': 5,
  'species\\LLA': 6,
  'species\\SCE': 7,
  'species\\SPO': 8,
  'species\\SPY': 9,
  'species\\SYN': 10,
  'species\\TMA': 11,
  'species\\YPE': 12},
 12)

## 1. GetMatrixNumber()

In [ ]:
#The Following matrix files must be in Folder
import os
def GetMatrixNumber():
    "This Function will Return the Matrix name choosed by User"
    import sys
    print ("\n1. BLOSUM45\n2. BLOSUM62\n3. BLOSUM82\n4. Quit")

    metrix_num = input("\nEnter a matrix number: ")
    if metrix_num not in ("1","2",'3'):
        print("Wrong input *%s*Sorry not in list\n"%metrix_num,"*"*20,"Good Bye","*"*20,"\n")
        sys.exit(1)
    if metrix_num =="1":
        return "BLOSUM45"
    if metrix_num == "2":
        return "BLOSUM62"
    if metrix_num == "3":
        return "BLOSUM82"
#GetMatrixNumber()

### 2. GetQuerySequence(genome)

In [82]:
species = "./species/"
def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed"""
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
        return gene_sequence_list                        
                                            
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err)    )
b = GetQuerySequence("AAE")

In [84]:
b[0]

'>gi|15605613|ref|NP_212986.1| elongation factor EF-G [Aquifex aeolicus VF5]\nMAREVPIEKLRNIGIVAHIDAGKTTTTERILYYTGKTYKIGEVHEGAATMDWMPQEKERGITITVATTAC\nYWTRNGERYQINIIDTPGHVDFSVEVVRSMKVLDGIVFIFSAVEGVQPQSEANWRWADRFQVPRIAFINK\nMDRLGADFYRVFKEIEEKLTIKPVAIQIPLGAEDQFEGVIDLMEMKAIRWLEETLGAKYEVVDIPPEYQE\nKAQEWREKMIETIVETDDELMEKYLEGQEISIDELRKALRKATIERKLVPVLCGSAFKNKGVQPLLDAVI\nDYLPSPIDLPPVKGTNPKTGEEEVRHPSDDEPFCAYAFKVMSDPYAGQLTYIRVFSGTLKAGSYVYNATK\nDEKQRAGRLLLMHANSREEIQQVSAGEICAVVGLDAATGDTLCDEKHPIILEKLEFPDPVISMAIEPKTK\nKDQEKLSQVLNKFMKEDPTFRATTDPETGQILIHGMGELHLEIMVDRMKREYGIEVNVGKPQVAYKETIR\nKKAIGEGKFIKQTGGRGQYGHAIIEIEPLPRGAGFEFIDDIHGGVIPKEFIPSVEKGVKEAMQNGILAGY\nPVVDVRVRLFDGSYHEVDSSDIAFQVAGSLAFKDAAKKADPVLLEPIMEVEVETPEKYVGDVIGDLNSRR\nGKIMGMENKGVITVIKAHVPLAEMFGYATTLRSLTQGRGTFIMKFSHYDEVPQQIAEKIIGERMAGKSS\n'

## 3.Write Query(query, parallel_num)

In [66]:
def WriteQuery(query, parallel_num):
    
    try:
        with open("./query"+"_"+str(parallel_num)+".csv", "w") as write_query:
            write_query.write(query)
    except IOError as err:
        print ("IOError occurred in WriteQuery function : " + str(err))

In [67]:
WriteQuery("krish is from ",10)

## 4. RunBlast( Subject , parallel_num)

In [1]:
! blastp -h

USAGE
  blastp [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-task task_name] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-negative_seqidlist filename]
    [-taxids taxids] [-negative_taxids taxids] [-taxidlist filename]
    [-negative_taxidlist filename] [-ipglist filename]
    [-negative_ipglist filename] [-entrez_query entrez_query]
    [-db_soft_mask filtering_algorithm] [-db_hard_mask filtering_algorithm]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]
    [-gapopen open_penalty] [-gapextend extend_penalty]
    [-qcov_hsp_perc float_value] [-max_hsps int_value]
    [-xdrop_ungap float_value] [-xdrop_gap float_value]
    [-xdrop_gap_final float_value] [-searchsp int_value] [-seg SEG_options]
    [-soft_masking soft_masking] [-matrix matrix_name]
    [-threshold float_value] [-c